<a href="https://colab.research.google.com/github/jus-tinian/vix_calc/blob/master/KoalasTD_SPXquotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create Working Env

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
!pip install -q findspark
!pip install -q pyspark

In [0]:
!pip install -q koalas

## Imports

In [0]:
import requests
import datetime

import pyspark
import databricks.koalas as ks

from pyspark.sql import SparkSession

## Create Spark Session

In [0]:
APP_NAME = "VIXcalc"
SPARK_URL = "local[*]"

In [0]:
spark = SparkSession.builder.master(SPARK_URL).appName(APP_NAME).getOrCreate()

In [9]:
spark

## Make Dates

In [0]:
today = datetime.datetime.today().date()

In [0]:
farDate = today + datetime.timedelta(days=37)

In [0]:
nearDate = today + datetime.timedelta(days=23)

## Get API response

In [0]:
# your key here
KEY = 'your key here'

In [0]:
url = f"https://api.tdameritrade.com/v1/marketdata/chains?apikey={KEY}&symbol=%24SPX.X&includeQuotes=TRUE&strategy=SINGLE&range=ALL&fromDate={str(nearDate)}&toDate={str(farDate)}"

In [0]:
req = requests.get(url)

In [0]:
reqJSON = req.json()

## Set Keys to access reqJSON

In [0]:
putKey = 'putExpDateMap'

In [0]:
callKey = 'callExpDateMap'

In [0]:
dateKeys = [i for i in list(req.json()[putKey].keys()) if datetime.datetime.strptime(i[:-3], '%Y-%m-%d').weekday() == 4]

In [0]:
nearKey = min(dateKeys)
farKey = max(dateKeys)

## Set Interest Rate

In [0]:
RATE = reqJSON['interestRate']

## Prep Pipe NO TOUCH

In [0]:
def from_json(jsonObj, sideKey, termKey):

  ks.set_option('compute.ops_on_diff_frames', True)

  # make list of strikes
  strikesList = list(jsonObj[sideKey][termKey].keys())
  
  keep = ['strikePrice', 'symbol', 'putCall', 'bid', 'ask', 'expirationDate', 'inTheMoney']

  source_dfs = [{k:v for k,v in reqJSON[sideKey][termKey][strike][0].items() if k in keep} for strike in strikesList]

  # similar to for loop to append all DFs into single DF
  source_df1 = ks.DataFrame(source_dfs)

  source_df2 = source_df1.loc[source_df1['bid'] != 0, :]

  standardCount = source_df2.loc[~source_df2['symbol'].str.contains('W'), :].shape[0]

  if standardCount != 0:
    source_df2 = source_df2.loc[~source_df2['symbol'].str.contains('W'), :]

  source_df2['expiration'] = ks.to_datetime(termKey[:-3]).date()

  source_df2 = source_df2.rename(columns={'expiration': 'expirationDate', 'expirationDate': 'expirationEpoch'})

  return source_df2.set_index('strikePrice')

# Near Puts

In [0]:
nearPuts = from_json(reqJSON, putKey, nearKey)

In [24]:
nearPuts.head()

,putCall,symbol,bid,ask,expirationEpoch,inTheMoney,expirationDate
strikePrice,,,,,,,
1350.0,PUT,SPXW_061220P1350,0.10,0.15,1591992000000,False,2020-06-12
1500.0,PUT,SPXW_061220P1500,0.05,0.20,1591992000000,False,2020-06-12
1550.0,PUT,SPXW_061220P1550,0.05,0.25,1591992000000,False,2020-06-12
1600.0,PUT,SPXW_061220P1600,0.10,0.25,1591992000000,False,2020-06-12
1650.0,PUT,SPXW_061220P1650,0.10,0.30,1591992000000,False,2020-06-12


## Far Puts

In [0]:
farPuts = from_json(reqJSON, putKey, farKey)

In [32]:
farPuts.head()

,putCall,symbol,bid,ask,expirationEpoch,inTheMoney,expirationDate
strikePrice,,,,,,,
900.0,PUT,SPX_061920P900,0.05,0.1,1592596800000,False,2020-06-19
1000.0,PUT,SPX_061920P1000,0.05,0.1,1592596800000,False,2020-06-19
1100.0,PUT,SPX_061920P1100,0.05,0.1,1592596800000,False,2020-06-19
1150.0,PUT,SPX_061920P1150,0.05,0.1,1592596800000,False,2020-06-19
1200.0,PUT,SPX_061920P1200,0.05,0.1,1592596800000,False,2020-06-19


# Near Calls

In [0]:
nearCalls = from_json(reqJSON, callKey, nearKey)

In [29]:
nearCalls.head()

,putCall,symbol,bid,ask,expirationEpoch,inTheMoney,expirationDate
strikePrice,,,,,,,
500.0,CALL,SPXW_061220C500,2453.4,2471.8,1591992000000,True,2020-06-12
600.0,CALL,SPXW_061220C600,2352.1,2370.4,1591992000000,True,2020-06-12
700.0,CALL,SPXW_061220C700,2258.1,2264.9,1591992000000,True,2020-06-12
800.0,CALL,SPXW_061220C800,2158.1,2165.0,1591992000000,True,2020-06-12
900.0,CALL,SPXW_061220C900,2058.2,2065.0,1591992000000,True,2020-06-12


## Far Calls

In [0]:
farCalls = from_json(reqJSON, callKey, farKey)

In [31]:
farCalls.head()

,putCall,symbol,bid,ask,expirationEpoch,inTheMoney,expirationDate
strikePrice,,,,,,,
100.0,CALL,SPX_061920C100,2852.5,2869.2,1592596800000,True,2020-06-19
200.0,CALL,SPX_061920C200,2752.6,2769.3,1592596800000,True,2020-06-19
300.0,CALL,SPX_061920C300,2653.2,2669.5,1592596800000,True,2020-06-19
400.0,CALL,SPX_061920C400,2552.6,2569.3,1592596800000,True,2020-06-19
500.0,CALL,SPX_061920C500,2452.6,2469.3,1592596800000,True,2020-06-19
